In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo

In [3]:
client = pymongo.MongoClient()

In [4]:
aus_realty_db = client.realtyaus
collection = aus_realty_db.aus_realty_data

In [5]:
aus_realty_df = pd.DataFrame(list(collection.find()))

In [6]:
aus_realty_df.head(2)

,_id,subdiv,address,data_type,status,status_extra,photo,time,mls_id,city,state,baths,sq_ft,acres,beds,zip,cost,source,id
0,5e112f17dbfd415408cf253e,Garden Villa Estates,3600 Villa Ct,Residential,Active,,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:13,6952847,Austin,TX,4.0,2326.0,0.13,4.0,78704,1059000.0,austin_realty,2019-12-21 17:20:13.006
1,5e112f17dbfd415408cf253f,Skyland Terrace,5405 Grover Ave,Residential,Active,,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:13,9195794,Austin,TX,3.0,2462.0,0.15,3.0,78756,825000.0,austin_realty,2019-12-21 17:20:13.009


In [7]:
aus_realty_df = aus_realty_df.drop_duplicates('mls_id')

In [8]:
aus_realty_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9436 entries, 0 to 9435
Data columns (total 19 columns):
_id             9436 non-null object
subdiv          9436 non-null object
address         9436 non-null object
data_type       9436 non-null object
status          9436 non-null object
status_extra    9436 non-null object
photo           9436 non-null object
time            9436 non-null datetime64[ns]
mls_id          9436 non-null int64
city            9436 non-null object
state           9436 non-null object
baths           9431 non-null float64
sq_ft           9432 non-null float64
acres           7044 non-null float64
beds            9424 non-null float64
zip             9436 non-null object
cost            9436 non-null float64
source          9436 non-null object
id              9436 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), object(11)
memory usage: 1.4+ MB


In [9]:
aus_df = aus_realty_df.query("city == 'Austin'")
aus_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2808 entries, 0 to 9433
Data columns (total 19 columns):
_id             2808 non-null object
subdiv          2808 non-null object
address         2808 non-null object
data_type       2808 non-null object
status          2808 non-null object
status_extra    2808 non-null object
photo           2808 non-null object
time            2808 non-null datetime64[ns]
mls_id          2808 non-null int64
city            2808 non-null object
state           2808 non-null object
baths           2807 non-null float64
sq_ft           2805 non-null float64
acres           2237 non-null float64
beds            2805 non-null float64
zip             2808 non-null object
cost            2808 non-null float64
source          2808 non-null object
id              2808 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), object(11)
memory usage: 438.8+ KB


In [ ]:
def central_tend_by_zipcode(df, sort='count', asc=False):
    grouped_zip = df.groupby('zip')
    data = pd.DataFrame({'median':grouped_zip['cost'].median().apply(lambda x : "{:,.2f}".format(x)),
                           'mean':grouped_zip['cost'].mean().apply(lambda x : "{:,.2f}".format(x)),
                           'count':grouped_zip['cost'].count(),
                            'mean_beds':grouped_zip['beds'].mean().apply(lambda x: "{:1.0f}".format(x)),
                            'mean_baths':grouped_zip['baths'].mean().apply(lambda x: "{:1.0f}".format(x)),
                            'mean_sqft':grouped_zip['sq_ft'].mean().apply(lambda x: "{:,.0f}".format(x))
                           })
    return data.sort_values(sort, ascending=asc)

In [ ]:
central_tend_by_zipcode(aus_df, sort='zip', asc=True)

In [ ]:
samp = aus_df[['address', 'city']].head()

In [11]:
from config import gkey
import requests

In [13]:
aus_df['addr'] = aus_df['address'] +', '+ aus_df['city']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
aus_df

In [28]:
# Google geocode to find lat, lng
#my_address = "3600 Villa Ct, Austin"
lt= []
ln= []
for address in endall['addr']:
    try:
        params = {
            "address": address,
            #"key": gkey}

        base_url = "https://maps.googleapis.com/maps/api/geocode/json"


        my_geo = requests.get(base_url, params).json()

        lt.append(my_geo["results"][0]["geometry"]["location"]["lat"])
        ln.append(my_geo["results"][0]["geometry"]["location"]["lng"])
        
    except:
        lt.append('not valid address')
        ln.append('not valid address')
        print(f'not working address: {address}')
        

not working address:  7807 Ryans Way, Austin
not working address:  7804 Ryans Way, Austin
not working address:  7915 Ryans Way, Austin
not working address:  7911 Ryans Way, Austin


In [ ]:
len(lt)

In [ ]:
len(ln)

In [ ]:
frth500['lat'] = lt
frth500['lon'] = ln

In [ ]:
samp['addr']= samp['address'] +' ' + samp['city']

In [ ]:
f400['lat']= lt
f400['lon']= ln

In [ ]:
f400 = aus_df[:400]

In [ ]:
svth400.tail()

In [22]:
endall = aus_df[2400:]

In [30]:
endall.to_csv('data/endall.csv', index=False)

In [29]:
endall['lat']=lt
endall['lon']=ln

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
sxth600.tail()

,_id,subdiv,address,data_type,status,status_extra,photo,time,mls_id,city,...,sq_ft,acres,beds,zip,cost,source,id,addr,lat,lon
7247,5e112f21dbfd415408cf418d,5200 Ave G Condo,5200 Avenue G #A,Residential,Pending,Taking Backups,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:30,1830771,Austin,...,1560.0,NaN,3.0,78751,695000.0,austin_realty,2019-12-21 17:20:30.001,"5200 Avenue G #A, Austin",30.316370,-97.721070
7250,5e112f21dbfd415408cf4190,Burnet Heights,1901 Romeria Dr,Residential,Pending,Taking Backups,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:14,8956441,Austin,...,2636.0,0.21,4.0,78757,695000.0,austin_realty,2019-12-21 17:20:14.008,"1901 Romeria Dr, Austin",30.335500,-97.735963
7251,5e112f21dbfd415408cf4191,"Steiner Ranch, Beverly Ridge",12429 Beverly Villas Ct,Residential,Pending,Taking Backups,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:15,3551473,Austin,...,3051.0,0.91,4.0,78732,695000.0,austin_realty,2019-12-21 17:20:15.003,"12429 Beverly Villas Ct, Austin",30.371546,-97.891113
7252,5e112f21dbfd415408cf4192,Rocky Creek,8804 FLYCATCHER Ct,Residential,Pending,Taking Backups,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:15,4227269,Austin,...,4058.0,0.24,5.0,78738,690000.0,austin_realty,2019-12-21 17:20:15.004,"8804 FLYCATCHER Ct, Austin",30.286533,-98.028663
7254,5e112f21dbfd415408cf4194,Great Hills,7713 Yaupon Dr,Residential,Pending,Taking Backups,https://rew-feed-images.global.ssl.fastly.net/...,2019-12-21 17:20:17,9324627,Austin,...,3302.0,0.23,4.0,78759,685000.0,austin_realty,2019-12-21 17:20:17.009,"7713 Yaupon Dr, Austin",30.417590,-97.779302


In [ ]:
sxth600.head()

In [24]:
len(endall)

408

In [ ]:
f400
nxt300
thr400
frth500
sxth600
svth400
endall

In [32]:
a=pd.read_csv('data/f400.csv')
b=pd.read_csv('data/nxt300.csv')
c=pd.read_csv('data/thr400.csv')
d=pd.read_csv('data/frth500.csv')
e=pd.read_csv('data/sxth600.csv')
f=pd.read_csv('data/svth400.csv')
g=pd.read_csv('data/endall.csv')



In [45]:
alldata=w.append(g)

In [56]:
clean_df = alldata[alldata['lat']!='not valid address']

In [59]:
clean_df['lat'] =clean_df['lat'].astype('float')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
clean_df['lon'] =clean_df['lon'].astype('float')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2799 entries, 0 to 407
Data columns (total 22 columns):
_id             2799 non-null object
subdiv          2798 non-null object
address         2799 non-null object
data_type       2799 non-null object
status          2799 non-null object
status_extra    838 non-null object
photo           2799 non-null object
time            2799 non-null object
mls_id          2799 non-null int64
city            2799 non-null object
state           2799 non-null object
baths           2798 non-null float64
sq_ft           2796 non-null float64
acres           2231 non-null float64
beds            2796 non-null float64
zip             2799 non-null int64
cost            2799 non-null float64
source          2799 non-null object
id              2799 non-null object
addr            2799 non-null object
lat             2799 non-null float64
lon             2799 non-null float64
dtypes: float64(7), int64(2), object(13)
memory usage: 502.9+ KB


In [62]:
collection = aus_realty_db.aus_realty_cleandf

In [63]:
def transform_data(row):
    data = row.to_dict()
    collection.insert_one(data)

clean_df.apply(transform_data, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
403    None
404    None
405    None
406    None
407    None
Length: 2799, dtype: object